In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import messagebox, ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.dates as mdates
import mplfinance as mpf
import threading
from scipy.optimize import minimize
import itertools


# Stock Data Fetching
def fetch_stock_data(ticker, period='1y', interval='1d'):
   try:
       if not ticker.endswith(".JK"):
           ticker += ".JK"
       stock_data = yf.download(ticker, period=period, interval=interval)
       stock_info = yf.Ticker(ticker).info
       return stock_data, stock_info
   except Exception as e:
       print(f"Error fetching data: {e}")
       return None, None


# Fill missing dates for line charts
def fill_missing_dates(data, interval='1d'):
   full_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='B')
   data = data.reindex(full_range)
   data.fillna(method='ffill', inplace=True)
   data.fillna(method='bfill', inplace=True)
   data.dropna(inplace=True)
   data.interpolate(method='linear', inplace=True)
   return data


# Stock Search Tab
def create_stock_search_ui(parent):
   tk.Label(parent, text="Enter Stock Ticker (e.g., TLKM, BBCA)").grid(row=0, column=0, padx=10, pady=10)
   stock_entry = tk.Entry(parent)
   stock_entry.grid(row=0, column=1, padx=10, pady=10)


   tk.Label(parent, text="Select Time Range").grid(row=1, column=0, padx=10, pady=10)
   time_range_var = tk.StringVar()
   time_range_dropdown = ttk.Combobox(parent, textvariable=time_range_var)
   time_range_dropdown['values'] = ['1 Day', '5 Days', '1 Month', '3 Months', '6 Months', '1 Year', '2 Years']
   time_range_dropdown.grid(row=1, column=1, padx=10, pady=10)
   time_range_dropdown.current(5)


   tk.Label(parent, text="Select Chart Type").grid(row=1, column=2, padx=10, pady=10)
   chart_type_var = tk.StringVar(value="Line")
   chart_type_dropdown = ttk.Combobox(parent, textvariable=chart_type_var)
   chart_type_dropdown['values'] = ['Line', 'Candlestick']
   chart_type_dropdown.grid(row=1, column=3, padx=10, pady=10)
   chart_type_dropdown.current(0)


   stock_info_label = tk.Label(parent, text="", font=("Arial", 12), justify='left')
   stock_info_label.grid(row=2, column=0, columnspan=4, padx=10, pady=10)


   style.use('ggplot')
   fig, ax = plt.subplots(figsize=(12, 7))
   canvas = FigureCanvasTkAgg(fig, master=parent)
   canvas.get_tk_widget().grid(row=3, column=0, columnspan=4)


   tooltip_label = tk.Label(parent, text="", font=("Arial", 12), bg="yellow", relief=tk.SOLID)
   tooltip_label.grid(row=4, column=0, columnspan=4, padx=10, pady=10)
   tooltip_label.place_forget()


   vline = ax.axvline(color='gray', linewidth=1, linestyle='--', visible=False)
   hline = ax.axhline(color='gray', linewidth=1, linestyle='--', visible=False)


   def get_period_and_interval(selection):
       if selection == '1 Day':
           return '1d', '5m'
       elif selection == '5 Days':
           return '5d', '1d'
       elif selection == '1 Month':
           return '1mo', '1d'
       elif selection == '3 Months':
           return '3mo', '1d'
       elif selection == '6 Months':
           return '6mo', '5d'
       elif selection == '1 Year':
           return '1y', '1mo'
       elif selection == '2 Years':
           return '2y', '1mo'
       else:
           return '1y', '1d'


   def plot_candlestick(data, ax, interval):
       if interval == '1mo':
           data_resampled = data.resample('M').agg({
               'Open': 'first',
               'High': 'max',
               'Low': 'min',
               'Close': 'last',
               'Volume': 'sum'
           })
       elif interval == '3mo':
           data_resampled = data.resample('3M').agg({
               'Open': 'first',
               'High': 'max',
               'Low': 'min',
               'Close': 'last',
               'Volume': 'sum'
           })
       else:
           data_resampled = data
       data_resampled.dropna(inplace=True)
       mpf.plot(data_resampled, type='candle', ax=ax, style='yahoo', volume=False,
                show_nontrading=True, datetime_format='%Y-%m-%d', figratio=(12,7), figscale=1.2)


   def plot_line_chart(data, ax):
       data = fill_missing_dates(data)
       ax.plot(data.index, data['Adj Close'], label="Price", color='blue', linewidth=2)
       ax.grid(True, linestyle='--', linewidth=0.5)


   def show_crosshair(event):
       if not event.inaxes:
           vline.set_visible(False)
           hline.set_visible(False)
           tooltip_label.place_forget()
           return


       vline.set_xdata(event.xdata)
       hline.set_ydata(event.ydata)
       vline.set_visible(True)
       hline.set_visible(True)


       date_str = mdates.num2date(event.xdata).strftime('%Y-%m-%d %H:%M:%S')
       tooltip_label.config(text=f"Date: {date_str}\nPrice: {event.ydata:.2f} IDR")


       widget_width = tooltip_label.winfo_reqwidth()
       widget_height = tooltip_label.winfo_reqheight()
       window_width = parent.winfo_width()
       window_height = parent.winfo_height()


       x_position = event.x + 20
       y_position = event.y + 20


       if x_position + widget_width > window_width:
           x_position = window_width - widget_width - 20
       if y_position + widget_height > window_height:
           y_position = window_height - widget_height - 20


       tooltip_label.place(x=x_position, y=y_position, anchor="nw")
       fig.canvas.draw_idle()


   def search_stock():
       ticker = stock_entry.get().upper()
       if not ticker:
           messagebox.showerror("Input Error", "Please enter a stock ticker.")
           return


       period, interval = get_period_and_interval(time_range_var.get())


       def fetch_and_display_data():
           stock_data, stock_info = fetch_stock_data(ticker, period, interval)
           if stock_data is None or stock_info is None:
               messagebox.showerror("Error", "Failed to fetch stock data.")
               return


           sector = stock_info.get('sector', 'N/A')
           industry = stock_info.get('industry', 'N/A')
           current_price = stock_info.get('regularMarketPrice', 'N/A')
           previous_close = stock_info.get('previousClose', 'N/A')


           stock_info_text = (f"Name: {stock_info.get('shortName', 'N/A')}\n"
                              f"Sector: {sector}\n"
                              f"Industry: {industry}\n"
                              f"Current Price: {current_price}\n"
                              f"Previous Close: {previous_close}")
           stock_info_label.config(text=stock_info_text)


           ax.clear()
           if chart_type_var.get() == 'Candlestick':
               plot_candlestick(stock_data, ax, interval)
           else:
               plot_line_chart(stock_data, ax)


           fig.autofmt_xdate()
           ax.set_title(f"{stock_info.get('shortName', 'Stock')} Historical Data ({time_range_var.get()})", fontsize=16, fontweight='bold')
           ax.set_xlabel("Date", fontsize=14)
           ax.set_ylabel("Price (IDR)", fontsize=14)
           ax.legend(fontsize=12)
           canvas.draw()


       threading.Thread(target=fetch_and_display_data).start()


   canvas.mpl_connect('motion_notify_event', show_crosshair)
   search_button = tk.Button(parent, text="Search Stock", command=search_stock)
   search_button.grid(row=0, column=2, padx=10, pady=10)


# Portfolio Management Tab
def create_portfolio_management_ui(parent):
   """
   Creates the portfolio management UI within the given parent frame.
   """
   # Input fields for stock tickers, budget, and desired return
   tk.Label(parent, text="Enter Stock Tickers (comma separated):").grid(row=0, column=0, padx=10, pady=10)
   tickers_entry = tk.Entry(parent, width=30)
   tickers_entry.grid(row=0, column=1, padx=10, pady=10)


   tk.Label(parent, text="Enter Investment Budget (IDR):").grid(row=1, column=0, padx=10, pady=10)
   budget_entry = tk.Entry(parent, width=20)
   budget_entry.grid(row=1, column=1, padx=10, pady=10)


   tk.Label(parent, text="Desired Portfolio Return (%)").grid(row=2, column=0, padx=10, pady=10)
   desired_return_entry = tk.Entry(parent, width=10)
   desired_return_entry.grid(row=2, column=1, padx=10, pady=10)


   result_label = tk.Label(parent, text="", font=("Arial", 12), justify='left')
   result_label.grid(row=5, column=0, columnspan=2, padx=10, pady=10)


   def fetch_multiple_stock_data(tickers, period="1y"):
       stock_data = {}
       valid_tickers = []
       for ticker in tickers:
           data = yf.download(ticker + ".JK", period=period)['Adj Close']
           if not data.empty:
               stock_data[ticker] = data
               valid_tickers.append(ticker)
       return pd.DataFrame(stock_data), valid_tickers


   def calculate_portfolio_metrics(stock_data):
       returns = stock_data.pct_change().mean() * 252  # Annualized returns
       cov_matrix = stock_data.pct_change().cov() * 252  # Annualized covariance
       return returns, cov_matrix


   def calculate_portfolio_performance(weights, returns, cov_matrix):
       portfolio_return = np.dot(weights, returns)
       portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
       portfolio_std_dev = np.sqrt(portfolio_variance)
       return portfolio_return, portfolio_std_dev


   def calculate_sharpe_ratio(portfolio_return, portfolio_std_dev, risk_free_rate=0.048):
       return (portfolio_return - risk_free_rate) / portfolio_std_dev


   def optimize_portfolio(returns, cov_matrix, desired_return):
       num_assets = len(returns)
       def portfolio_risk(weights):
           return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))


       constraints = (
           {"type": "eq", "fun": lambda weights: np.sum(weights) - 1},
           {"type": "eq", "fun": lambda weights: np.dot(weights, returns) - desired_return}
       )
       bounds = [(0, 1) for _ in range(num_assets)]
       initial_weights = [1 / num_assets] * num_assets


       result = minimize(portfolio_risk, initial_weights, method="SLSQP", bounds=bounds, constraints=constraints)
       return result.x if result.success else None


   def knapsack_adjustment_in_lots(weights, stock_prices, budget):
       stock_allocations = weights * budget
       stock_lots = (stock_allocations // (stock_prices * 100)).astype(int)
       final_allocations = stock_lots * stock_prices * 100
       total_spent = np.sum(final_allocations)
       return stock_lots, final_allocations, total_spent


   def plot_allocation_pie_chart(weights, tickers):
       pie_window = tk.Toplevel()
       pie_window.title("Portfolio Allocation")
       fig, ax = plt.subplots(figsize=(6, 6))
       ax.pie(weights, labels=tickers, autopct='%1.1f%%', startangle=90, shadow=True)
       ax.axis('equal')
       canvas = FigureCanvasTkAgg(fig, master=pie_window)
       canvas.get_tk_widget().pack()
       canvas.draw()


   def provide_recommendations(portfolio_return, portfolio_std_dev, sharpe_ratio, max_return):
       recommendations = []
       if sharpe_ratio < 0.5:
           recommendations.append("The Sharpe ratio is relatively low. Consider adjusting stock selection.")
       elif sharpe_ratio > 1.0:
           recommendations.append("The portfolio has a good Sharpe ratio, indicating strong performance.")
       if portfolio_std_dev > 0.20:
           recommendations.append(f"The portfolio risk (Std Dev) is high ({portfolio_std_dev*100:.2f}%).")
       else:
           recommendations.append(f"The portfolio risk (Std Dev) is moderate ({portfolio_std_dev*100:.2f}%).")
       recommendations.append(f"The maximum achievable return is {max_return*100:.2f}%. Adjust expectations if needed.")
       return "\n".join(recommendations)


   def display_portfolio_results():
       try:
           tickers = [ticker.strip().upper() for ticker in tickers_entry.get().split(',')]
           budget = float(budget_entry.get())
           desired_return = float(desired_return_entry.get()) / 100


           stock_data, valid_tickers = fetch_multiple_stock_data(tickers)
           stock_prices = stock_data.iloc[-1].values
           returns, cov_matrix = calculate_portfolio_metrics(stock_data)
           max_return = max(returns)


           if desired_return > max_return:
               messagebox.showerror("Error", f"Desired return too high. Max achievable: {max_return*100:.2f}%.")
               return


           weights = optimize_portfolio(returns, cov_matrix, desired_return)
           if weights is None:
               result_label.config(text="Error: Desired return not achievable.")
               return


           portfolio_return, portfolio_std_dev = calculate_portfolio_performance(weights, returns, cov_matrix)
           sharpe_ratio = calculate_sharpe_ratio(portfolio_return, portfolio_std_dev)


           stock_lots, final_allocations, total_spent = knapsack_adjustment_in_lots(weights, stock_prices, budget)


           allocation_text = "\n".join([f"{valid_ticker}: {stock_lots[i]} lots (Rp {final_allocations[i]:,.2f})"
                                        for i, valid_ticker in enumerate(valid_tickers)])
           recommendations = provide_recommendations(portfolio_return, portfolio_std_dev, sharpe_ratio, max_return)


           result_text = (
               f"Optimal Portfolio Allocation:\n{allocation_text}\n\n"
               f"Expected Portfolio Return: {portfolio_return*100:.2f}%\n"
               f"Portfolio Risk (Std Dev): {portfolio_std_dev:.2f}\n"
               f"Sharpe Ratio: {sharpe_ratio:.2f}\n\n"
               f"Total Spent from Budget: Rp {total_spent:,.2f}\n\n"
               f"Recommendations:\n{recommendations}"
           )
           result_label.config(text=result_text)


           plot_allocation_pie_chart(final_allocations / total_spent, valid_tickers)
       except ValueError:
           messagebox.showerror("Input Error", "Please ensure all inputs are correct.")


   optimize_button = tk.Button(parent, text="Optimize Portfolio", command=display_portfolio_results)
   optimize_button.grid(row=4, column=0, columnspan=2, pady=10)


def fetch_backtest_data(ticker, timeframe):
   timeframe = timeframe.lower()
   if timeframe == '1 year':
       data = yf.download(ticker, period="1y")
   elif timeframe == '6 months':
       data = yf.download(ticker, period="6mo")
   elif timeframe == '2 years':
       data = yf.download(ticker, period="2y")
   elif timeframe == '5 years':
       data = yf.download(ticker, period="5y")
   elif timeframe == 'ytd':
       data = yf.download(ticker, period="ytd")
   else:
       data = yf.download(ticker, period="max")
   return data


# Optimized Indicators
def calculate_sma(data, short_window=10, long_window=50):
   data['SMA10'] = data['Close'].rolling(window=short_window).mean()
   data['SMA50'] = data['Close'].rolling(window=long_window).mean()
   data['SMA_Signal'] = np.where(data['SMA10'] > data['SMA50'], 1, -1)
   return data


def calculate_ema(data, short_window=20, long_window=50):
   data['EMA20'] = data['Close'].ewm(span=short_window, adjust=False).mean()
   data['EMA50'] = data['Close'].ewm(span=long_window, adjust=False).mean()
   data['EMA_Signal'] = np.where(data['EMA20'] > data['EMA50'], 1, -1)
   return data


def calculate_rsi(data, window=14):
   delta = data['Close'].diff()
   gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
   loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
   rs = gain / loss
   data['RSI'] = 100 - (100 / (1 + rs))
   data['RSI_Signal'] = np.where(data['RSI'] < 30, 1, np.where(data['RSI'] > 70, -1, 0))
   return data


def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
   data['EMA12'] = data['Close'].ewm(span=short_window, adjust=False).mean()
   data['EMA26'] = data['Close'].ewm(span=long_window, adjust=False).mean()
   data['MACD'] = data['EMA12'] - data['EMA26']
   data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
   data['MACD_Signal'] = np.where(data['MACD'] > data['Signal_Line'], 1, -1)
   return data


def calculate_bollinger_bands(data, window=20, num_std=2):
   data['SMA20'] = data['Close'].rolling(window=window).mean()
   data['Upper'] = data['SMA20'] + (data['Close'].rolling(window=window).std() * num_std)
   data['Lower'] = data['SMA20'] - (data['Close'].rolling(window=window).std() * num_std)
   data['Bollinger_Bands_Signal'] = np.where(data['Close'] < data['Lower'], 1, np.where(data['Close'] > data['Upper'], -1, 0))
   return data


def calculate_obv(data):
   data['OBV'] = (np.sign(data['Close'].diff()) * data['Volume']).fillna(0).cumsum()
   data['OBV_Signal'] = np.where(data['OBV'] > data['OBV'].rolling(window=7).mean(), 1, -1)
   return data


# Run backtest with combined signals and track equity
def run_backtest(data, indicators):
   signals = [f'{indicator}_Signal' for indicator in indicators]
   data['Final_Signal'] = data[signals].mean(axis=1).apply(lambda x: 1 if x > 0 else -1)


   data['Returns'] = data['Close'].pct_change() * data['Final_Signal'].shift(1)
   data['Equity_Curve'] = (1 + data['Returns']).cumprod()
   data['Cumulative_Max'] = data['Equity_Curve'].cummax()
   data['Drawdown'] = (data['Cumulative_Max'] - data['Equity_Curve']) / data['Cumulative_Max']


   max_drawdown = data['Drawdown'].max()
   net_profit = (data['Equity_Curve'].iloc[-1] - 1) * 100
   excess_returns = data['Returns'] - (0.048 / 252)  # Adjust daily returns for 4.8% annual risk-free rate
   sharpe_ratio = excess_returns.mean() / excess_returns.std() * np.sqrt(252)
   volatility = data['Returns'].std() * np.sqrt(252)
   profit_factor = (data['Returns'][data['Returns'] > 0].sum()) / abs(data['Returns'][data['Returns'] < 0].sum())
  
   return net_profit, sharpe_ratio, max_drawdown * 100, volatility, profit_factor


# Suggest best combination based on defined criteria
def suggest_best_combination(results):
   df = pd.DataFrame(results, columns=['Combination', 'Net Profit', 'Sharpe Ratio', 'Max Drawdown', 'Volatility', 'Profit Factor'])
   valid_combinations = df[(df['Sharpe Ratio'] > 2) & (df['Max Drawdown'] < 20)]
  
   if not valid_combinations.empty:
       best_combination = valid_combinations.loc[valid_combinations['Net Profit'].idxmax()]['Combination']
   else:
       best_combination = df.loc[df['Sharpe Ratio'].idxmax()]['Combination']
  
   return best_combination


# Strategy Functions
def basic_strategy(data):
   position = 0
   buy_price = 0
   returns = []


   for i in range(1, len(data)):
       signal = data['Final_Signal'].iloc[i]
       price = data['Close'].iloc[i]


       if signal == 1 and position == 0:
           position = 1
           buy_price = price
       elif signal == -1 and position == 1:
           returns.append(price / buy_price - 1)
           position = 0


   return np.sum(returns)


def accumulation_strategy(data):
   position = 0
   buy_prices = []
   returns = []


   for i in range(1, len(data)):
       signal = data['Final_Signal'].iloc[i]
       price = data['Close'].iloc[i]


       if signal == 1:
           buy_prices.append(price)
           position += 1
       elif signal == -1 and position > 0:
           total_buy_price = sum(buy_prices) / position
           returns.append(price / total_buy_price - 1)
           buy_prices.clear()
           position = 0


   return np.sum(returns)


def profit_locking_strategy(data):
   position = 0
   buy_price = 0
   returns = []


   for i in range(1, len(data)):
       signal = data['Final_Signal'].iloc[i]
       price = data['Close'].iloc[i]


       if signal == 1 and position == 0:
           position = 1
           buy_price = price
       elif signal == -1 and position == 1 and price > buy_price:
           returns.append(price / buy_price - 1)
           position = 0


   return np.sum(returns)


# Evaluate all strategies with the best combination
def evaluate_strategies(data, best_combination):
   signals = [f'{indicator}_Signal' for indicator in best_combination]
   data['Final_Signal'] = data[signals].mean(axis=1).apply(lambda x: 1 if x > 0 else -1)


   strategy_results = {
       'Basic Strategy': basic_strategy(data),
       'Accumulation Strategy': accumulation_strategy(data),
       'Profit Locking Strategy': profit_locking_strategy(data)
   }


   return strategy_results


# Display equity curve for the best strategy
def display_equity_curve(data, indicators, best_strategy, ax):
   signals = [f'{indicator}_Signal' for indicator in indicators]
   data['Final_Signal'] = data[signals].mean(axis=1).apply(lambda x: 1 if x > 0 else -1)


   position = 0
   buy_price = 0
   equity_curve = [1]


   for i in range(1, len(data)):
       signal = data['Final_Signal'].iloc[i]
       price = data['Close'].iloc[i]
      
       if best_strategy == 'Basic Strategy':
           if signal == 1 and position == 0:
               position = 1
               buy_price = price
           elif signal == -1 and position == 1:
               equity_curve.append(equity_curve[-1] * (price / buy_price))
               position = 0
           else:
               equity_curve.append(equity_curve[-1])


       elif best_strategy == 'Accumulation Strategy':
           if signal == 1:
               equity_curve.append(equity_curve[-1] * (1 + price / equity_curve[-1]))
           elif signal == -1 and position > 0:
               total_buy_price = sum(buy_price) / position if position > 0 else buy_price
               equity_curve.append(equity_curve[-1] * (price / total_buy_price))
               position = 0
           else:
               equity_curve.append(equity_curve[-1])


       elif best_strategy == 'Profit Locking Strategy':
           if signal == 1 and position == 0:
               position = 1
               buy_price = price
           elif signal == -1 and position == 1 and price > buy_price:
               equity_curve.append(equity_curve[-1] * (price / buy_price))
               position = 0
           else:
               equity_curve.append(equity_curve[-1])


   while len(equity_curve) < len(data):
       equity_curve.append(equity_curve[-1])


   data['Equity_Curve'] = pd.Series(equity_curve, index=data.index)


   # Plot the equity curve on the provided axis
   ax.plot(data.index, data['Equity_Curve'], label=f'Equity Curve ({best_strategy})', color='red')
   ax.set_title(f'Equity Curve for Best Strategy: {best_strategy}')
   ax.legend()
   ax.grid(True)




#####


####


# Display trade curve for the best strategy


def display_trade_curve(data, indicators, best_strategy, ax):
   # Calculate the final signal from the indicators
   signals = [f'{indicator}_Signal' for indicator in indicators]
   data['Final_Signal'] = data[signals].mean(axis=1).apply(lambda x: 1 if x > 0 else -1)


   position = 0
   buy_price = 0


   # Iterate through data to determine buy/sell signals
   for i in range(1, len(data)):
       signal = data['Final_Signal'].iloc[i]
       price = data['Close'].iloc[i]


       if best_strategy == 'Basic Strategy':
           if signal == 1 and position == 0:  # Buy signal
               position = 1
               buy_price = price
               ax.annotate('↑', (data.index[i], price), textcoords="offset points", xytext=(0, 10),
                           ha='center', color='green', fontsize=12, weight='bold')
           elif signal == -1 and position == 1:  # Sell signal
               ax.annotate('↓', (data.index[i], price), textcoords="offset points", xytext=(0, -10),
                           ha='center', color='red', fontsize=12, weight='bold')
               position = 0


       elif best_strategy == 'Accumulation Strategy':
           if signal == 1:  # Buy signal
               ax.annotate('↑', (data.index[i], price), textcoords="offset points", xytext=(0, 10),
                           ha='center', color='green', fontsize=12, weight='bold')
           elif signal == -1 and position > 0:  # Sell signal
               ax.annotate('↓', (data.index[i], price), textcoords="offset points", xytext=(0, -10),
                           ha='center', color='red', fontsize=12, weight='bold')
               position = 0


       elif best_strategy == 'Profit Locking Strategy':
           if signal == 1 and position == 0:  # Buy signal
               position = 1
               buy_price = price
               ax.annotate('↑', (data.index[i], price), textcoords="offset points", xytext=(0, 10),
                           ha='center', color='green', fontsize=12, weight='bold')
           elif signal == -1 and position == 1 and price > buy_price:  # Sell signal
               ax.annotate('↓', (data.index[i], price), textcoords="offset points", xytext=(0, -10),
                           ha='center', color='red', fontsize=12, weight='bold')
               position = 0


   # Plot the stock price on the Y-axis (close price) over time (X-axis)
   ax.plot(data.index, data['Close'], label='Stock Price', color='blue', linewidth=1)
   ax.set_title(f'Buy/Sell Signal Curve for Best Strategy: {best_strategy}')
   ax.legend()
   ax.grid(True)














#####




#####




# Main function


def create_decision_table(results):
   df = pd.DataFrame(results, columns=['Indicator Combination', 'Net Profit (%)', 'Sharpe Ratio', 'Max Drawdown (%)', 'Volatility', 'Profit Factor'])
  
   # Find the best combination based on a criterion (e.g., Sharpe Ratio or Net Profit)
   best_combination = df.loc[df['Sharpe Ratio'].idxmax()]
  
   # Filter for individual indicators (single-element combinations)
   single_indicators = df[df['Indicator Combination'].apply(lambda x: len(x) == 1)]
  
   # Concatenate the single indicators and best combination into a final table
   final_df = pd.concat([single_indicators, pd.DataFrame([best_combination])])
  
   # Reset index for a clean display
   final_df.reset_index(drop=True, inplace=True)
  
   return final_df


# Prediction based on selected indicator combination
def predict_next_day(data, best_combination, days_ahead=1):
   signals = [f'{indicator}_Signal' for indicator in best_combination]
   data['Final_Signal'] = data[signals].mean(axis=1).apply(lambda x: 1 if x > 0 else -1)
   return data['Final_Signal'].iloc[-days_ahead]




def create_backtesting_analysis_ui(parent):
   tk.Label(parent, text="Enter Stock Ticker (e.g., BBCA):").grid(row=0, column=0, padx=10, pady=10)
   ticker_entry = tk.Entry(parent)
   ticker_entry.grid(row=0, column=1, padx=10, pady=10)


   tk.Label(parent, text="Select Timeframe:").grid(row=1, column=0, padx=10, pady=10)
   timeframe_var = tk.StringVar()
   timeframe_dropdown = ttk.Combobox(parent, textvariable=timeframe_var)
   timeframe_dropdown['values'] = ['1 Year', '6 Months', '2 Years', '5 Years', 'YTD']
   timeframe_dropdown.grid(row=1, column=1, padx=10, pady=10)
   timeframe_dropdown.current(0)


   def run_backtesting():
       ticker = ticker_entry.get().upper()
       if not ticker.endswith(".JK"):
           ticker += ".JK"


       timeframe = timeframe_var.get()
       data = fetch_backtest_data(ticker, timeframe)
      
       # Calculate indicators and perform backtesting as per the provided code
       indicators = ['SMA', 'EMA', 'RSI', 'MACD', 'Bollinger_Bands', 'OBV']
      
       # Calculate all indicators as per your code
       data = calculate_sma(data)
       data = calculate_ema(data)
       data = calculate_rsi(data)
       data = calculate_macd(data)
       data = calculate_bollinger_bands(data)
       data = calculate_obv(data)
      
       # Run backtest with all indicator combinations
       all_combinations = list(itertools.chain.from_iterable(itertools.combinations(indicators, r) for r in range(1, len(indicators) + 1)))
       results = []


       for combo in all_combinations:
           net_profit, sharpe_ratio, max_drawdown, volatility, profit_factor = run_backtest(data, combo)
           results.append([combo, net_profit, sharpe_ratio, max_drawdown, volatility, profit_factor])


       best_combination = suggest_best_combination(results)


       # Evaluate strategies and display the best strategy results
       strategy_results = evaluate_strategies(data, best_combination)
       best_strategy = max(strategy_results, key=strategy_results.get)


       result_text = f"Best Strategy: {best_strategy}\n"
       for strategy, result in strategy_results.items():
           result_text += f"{strategy}: Total Return = {result:.2f}\n"
       result_label.config(text=result_text)


       # Display the equity curve in a new window
       def show_equity_curve():
           curve_window = tk.Toplevel(parent)
           curve_window.title("Equity Curve")


   # Set up the figure and axis for plotting
           fig, ax = plt.subplots(figsize=(10, 6))
  
   # Call the function to plot the equity curve directly on the provided `ax`
           display_equity_curve(data, best_combination, best_strategy, ax)  # Pass ax to display_equity_curve


   # Embed the figure in the Tkinter window using FigureCanvasTkAgg
           canvas = FigureCanvasTkAgg(fig, master=curve_window)
           canvas.get_tk_widget().pack()
           canvas.draw()


       # Display the TRADE curve in a new window
       def show_trade_curve():
           curve_window = tk.Toplevel(parent)
           curve_window.title("Trade Curve")


   # Set up the figure and axis for plotting
           fig, ax = plt.subplots(figsize=(10, 8))
  
   # Call the function to plot the equity curve directly on the provided `ax`
           display_trade_curve(data, best_combination, best_strategy, ax)  # Pass ax to display_equity_curve


   # Embed the figure in the Tkinter window using FigureCanvasTkAgg
           canvas = FigureCanvasTkAgg(fig, master=curve_window)
           canvas.get_tk_widget().pack()
           canvas.draw()




       # Display the decision table in a larger window
       def show_decision_table():
           table_window = tk.Toplevel(parent)
           table_window.title("Decision Table")
           table_window.geometry("800x600")  # Adjusted to make the decision table window larger
           decision_table_df = create_decision_table(results)


           # Creating a Text widget to display the DataFrame in the GUI
           text_widget = tk.Text(table_window, wrap="none")
           text_widget.insert(tk.END, decision_table_df.to_string(index=False))
           text_widget.pack(fill="both", expand=True)


       # Create buttons for equity curve and decision table
       equity_button = tk.Button(parent, text="Show Equity Curve", command=show_equity_curve)
       equity_button.grid(row=5, column=0, padx=10, pady=10)


       decision_button = tk.Button(parent, text="Show Decision Table", command=show_decision_table)
       decision_button.grid(row=5, column=3, padx=10, pady=10)


       trade_button = tk.Button(parent, text="Show Trade Curve", command=show_trade_curve)
       trade_button.grid(row=6, column=1, padx=10, pady=10)


      
   result_label = tk.Label(parent, text="", font=("Arial", 12), justify='left')
   result_label.grid(row=4, column=0, columnspan=2, padx=10, pady=10)


   backtest_button = tk.Button(parent, text="Run Backtesting", command=run_backtesting)
   backtest_button.grid(row=2, column=0, columnspan=2, pady=10)




# Main GUI Setup
def create_gui():
   root = tk.Tk()
   root.title("Stock Search and Portfolio Optimizer")


   notebook = ttk.Notebook(root)
   notebook.pack(fill='both', expand=True)


   stock_search_frame = ttk.Frame(notebook)
   notebook.add(stock_search_frame, text="Stock Search")
   create_stock_search_ui(stock_search_frame)


   portfolio_management_frame = ttk.Frame(notebook)
   notebook.add(portfolio_management_frame, text="Portfolio Management")
   create_portfolio_management_ui(portfolio_management_frame)
  
   backtesting_analysis_frame = ttk.Frame(notebook)
   notebook.add(backtesting_analysis_frame, text="Backtesting Analysis")
   create_backtesting_analysis_ui(backtesting_analysis_frame)


   root.mainloop()


if __name__ == "__main__":
   create_gui()












<IPython.core.display.Javascript object>

2025-02-12 09:27:29.528 python[61163:168333] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-12 09:27:29.529 python[61163:168333] +[IMKInputSession subclass]: chose IMKInputSession_Modern
/var/folders/nm/8w57qqd567d_k26lq5m4klfw0000gp/T/ipykernel_61163/3145443009.py:139: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  vline.set_xdata(event.xdata)
/var/folders/nm/8w57qqd567d_k26lq5m4klfw0000gp/T/ipykernel_61163/3145443009.py:140: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  hline.set_ydata(event.ydata)
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%**

<IPython.core.display.Javascript object>